In [ ]:
%load_ext autoreload

# 获取数据集

In [ ]:
import os
train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)

all_data = valdata + traindata

# 一些几何学工具

In [ ]:
import matplotlib.path as mpath
import matplotlib.pyplot as plt

from tianmoucv.isp.transform import compute_minimum_convex_hull,calculate_area,is_inside,interpolate_zero_point

def process_matrix(matrix):
    vertices = compute_minimum_convex_hull(matrix)
    print("闭包顶点：", vertices)

    # 计算闭包面积
    area = calculate_area(vertices)
    print("闭包面积：", area)

    matrix_inver = matrix.copy()
    matrix_inver[matrix!=0] = 0
    matrix_inver[matrix==0] = 1
    matrix_inver = matrix_inver[:-2,:-2]
    zero_points = np.transpose(np.nonzero(matrix_inver))
    
    # 判断零点是否落在闭包内部
    matrix_output = matrix.copy()
    for zero_point in zero_points:
        if is_inside(vertices, zero_point):
            #print("零点在闭包内")
            # 进行插值
            interpolated_value = interpolate_zero_point(matrix, vertices, zero_point)
            #print("插值结果：", interpolated_value,zero_point)
            matrix_output[zero_point[0],zero_point[1]] = interpolated_value
        else:
            pass
            #print("零点不在闭包内")
    return matrix_output

# 可视化表示结果

In [ ]:
%autoreload
import sys,os
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import math,time
import matplotlib.pyplot as plt

from tianmoucv.isp import fourdirection2xy
from tianmoucv.proc.reconstruct import poisson_blend
from tianmoucv.data import TianmoucDataReader

from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D

key_list = ['test_exam_fan4']


for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500, matchkey=key,speedUpRate=1)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index == 0:
            F0 = sample['F0'][0,...]
            F1 = sample['F1'][0,...]
            tsdiff = sample['tsdiff'][0,...]
            print(F0.shape,tsdiff.shape)
            print('帧表示')
            plt.figure(figsize=(10,5))
            plt.subplot(1,2,1)
            plt.imshow(F0.numpy())
            plt.subplot(1,2,2)
            plt.imshow(tsdiff[:,12,...].permute(1,2,0).numpy())
            plt.show()
            print('时空表面')
            edgesurface = np.zeros([320,640,3])
            thresh = 0.1
            for t in range(tsdiff.shape[1]):
                mask = abs(tsdiff[0,t,...])>thresh
                edgesurface[...,0][mask] = t/25.0
                edgesurface[...,1][mask] = tsdiff[1,t,...][mask]
                edgesurface[...,2][mask] = tsdiff[2,t,...][mask]
                
            fig = plt.figure(figsize=(10,5))
            plt.subplot(1,1,1)
            plt.imshow(edgesurface)
            plt.show()
            fig = plt.figure(figsize=(10,10))
            
            # 创建数据
            x = np.linspace(0, 640, 640)
            y = np.linspace(0, 320, 320)
            X, Y = np.meshgrid(x, y)
            Z = edgesurface[...,0]
            
            # 绘制三维表面
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X, Y, Z, cmap='viridis')

            # 在表面上绘制二维箭头
            U = edgesurface[...,1]
            V = edgesurface[...,2]
            W = np.zeros_like(U)
            length = np.sqrt(V**2+U**2)

            ax.quiver(X, Y, Z, U, V, W, length = 1 )
            # 设置图形参数
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            ax.view_init(azim=-225, elev=30) 
            plt.show()
            print('三维矢量云')
            
            
        break